In [1]:
# requirements:
! pip install huggingface_hub
! git config --global credential.helper store

  Using cached huggingface_hub-0.10.1-py3-none-any.whl (163 kB)
  Using cached filelock-3.8.0-py3-none-any.whl (10 kB)


In [45]:
import os
import json
import numpy as np
from collections import defaultdict
import copy
import time
import json

In [2]:
from huggingface_hub import notebook_login
from huggingface_hub import HfFolder


#enter your API key, you can make one for free on HF
notebook_login()

In [7]:
from huggingface_hub import InferenceApi

inference = InferenceApi("bigscience/bloom",token=HfFolder.get_token())

In [8]:
def infer(prompt,
          max_length = 32,
          top_k = 0,
          num_beams = 0,
          no_repeat_ngram_size = 2,
          top_p = 0.85,
          seed=42,
          temperature=0.7,
          greedy_decoding = False,
          return_full_text = False):
    

    top_k = None if top_k == 0 else top_k
    do_sample = False if num_beams > 0 else not greedy_decoding
    num_beams = None if (greedy_decoding or num_beams == 0) else num_beams
    no_repeat_ngram_size = None if num_beams is None else no_repeat_ngram_size
    top_p = None if num_beams else top_p
    early_stopping = None if num_beams is None else num_beams > 0

    params = {
        "max_new_tokens": max_length,
        "top_k": top_k,
        "top_p": top_p,
        "temperature": temperature,
        "do_sample": do_sample,
        "seed": seed,
        "early_stopping":early_stopping,
        "no_repeat_ngram_size":no_repeat_ngram_size,
        "num_beams":num_beams,
        "return_full_text":return_full_text
    }
    
    s = time.time()
    response = inference(prompt, params=params)
    #print(response)
    proc_time = time.time()-s
    #print(f"Processing time was {proc_time} seconds")
    return response

In [12]:
# prompt = "※ Mojim.com \n以下是周杰伦创作的歌词：\n" # Adding the Mojim characters to the data just extracts the lyrics from the model straight up
# prompt = "以下是周杰伦创作的歌词：\n"

#TOP_P = [0.80, 0.85, 0.90, 0.95]
TOP_P = 1.0
NUM_SAMPLES = 3000
MAX_LENGTH = 100
TEMP = 1.0
#prompt = "歌词：\n"
prompt = "下面是一首歌的中文歌词。\n歌词：\n"

START_S = 0
seeds = np.arange(START_S, NUM_SAMPLES)

In [14]:
samples = []

with open(f"bloom_log_{TOP_P}.txt", "a") as f:
    for s in seeds:
        success = False

        while not success:
            resp = infer(prompt, seed=int(s), max_length=MAX_LENGTH, top_p=TOP_P, temperature=TEMP)
            record = {"response":resp, "seed":int(s), "top_p":TOP_P, "max_length":MAX_LENGTH, "temperature":TEMP}
            print(json.dumps(record), file=f)

            try:
                a = resp[0]
                success = True
            except KeyError:
                print("Received error... waiting 15 seconds.", file=f)
                success = False
                time.sleep(15)

        samples.append(record)

In [ ]:
for s in samples:
    print("=====")
    print(s["response"][0]["generated_text"])

In [ ]:
print(resp[0]["generated_text"])

In [ ]:
# 说真的,你比从前快乐,那祝福的话叫我如何能够说的出口,过往的欢乐是否褪色想问你怎么舍得不要在耳边再说你会想我,如果我真的会难过,不是说好要一起到老,怎么会剩我一个人,度过风雨的时分,也许时间是一种解药,也是我现在正服下的毒药,说好要一起到老,怎么剩我一个人,度过风雨的时分,爱情给的苦药,我

In [ ]:
# 以下是周杰伦创作的歌词：
# 我听到传来的谁的声音
# 象那梦里呜咽中的小河
# 我看到远去的谁的步伐
# 遮住告别时哀伤的眼神
# 不明白的是为何你情愿
# 让风尘刻画你的样子
# 就像早已忘情的世界
# 曾经拥有你的名字我的声音
# 那悲歌总会在梦中清醒
# 诉说一点哀伤过的往事
# 那看似满不在乎转过身的
# 是风干泪眼后萧瑟的影子
# 不明白的是为何

This one looks like it was straight up lifted from Mojim.  NLM plagiarism is a problem here.

In [ ]:
# 以下是周杰伦创作的歌词：
# 《七里香》
# 作曲：周杰伦
# 作词：方文山
# 窗外的麻雀 在电线杆上多嘴
# 你说这一句 很有夏天的感觉
# 手中的铅笔 在纸上来来回回
# 我用几行字形容你是我的谁
# 秋刀鱼的滋味 猫跟你都想了解
# 初恋的香味就这样被我们寻回
# 那温暖的阳光 像刚摘的鲜艳草莓
# 你说你舍不得吃掉这一种感觉
# 雨

## Calculate metrics on generated data

In [26]:
import pandas as pd
from nltk import ngrams
import nltk

In [70]:
p080 = pd.read_json("./BLOOM/3K_BLOOM_p080.jsonl", lines=True)
p085 = pd.read_json("./BLOOM/3K_BLOOM_p085.jsonl", lines=True)
p090 = pd.read_json("./BLOOM/3K_BLOOM_p090.jsonl", lines=True)
p095 = pd.read_json("./BLOOM/3K_BLOOM_p095.jsonl", lines=True)
p099 = pd.read_json("./BLOOM/3K_BLOOM_p099.jsonl", lines=True)
rt = pd.read_json("./BLOOM/top_song_lyrics_cleaned_cn.jsonl", lines=True)

In [71]:
p080_text = p080["response"].apply(lambda a: a[0]["generated_text"]).str.split("下面是一首歌的中文歌词。\n歌词：\n").apply(lambda a: a[1])
p085_text = p085["response"].apply(lambda a: a[0]["generated_text"]).str.split("下面是一首歌的中文歌词。\n歌词：\n").apply(lambda a: a[1])
p090_text = p090["response"].apply(lambda a: a[0]["generated_text"]).str.split("下面是一首歌的中文歌词。\n歌词：\n").apply(lambda a: a[1])
p095_text = p095["response"].apply(lambda a: a[0]["generated_text"]).str.split("下面是一首歌的中文歌词。\n歌词：\n").apply(lambda a: a[1])
p099_text = p099["response"].apply(lambda a: a[0]["generated_text"]).str.split("下面是一首歌的中文歌词。\n歌词：\n").apply(lambda a: a[1])

In [73]:
real_text = pd.Series([a[:100] for a in rt["cleaned_lyrics"]])

In [76]:
"""
Simplified from:
References: https://github.com/wangpf3/imagine-and-verbalize/blob/main/verbalization_learning/lib/utils/text_evaluation.py
https://github.com/VegB/iNLG/blob/main/code/text_evaluation.py
"""

def evaluator(gts, res):
    eval = {}
    # =================================================
    # Compute scores
    # =================================================
    for scorer, method in scorers:
        score, scores = scorer.compute_score(gts, res)
        if type(method) == list:
            for sc, scs, m in zip(score, scores, method):
                eval[m] = sc
        else:
            eval[method] = score
    return eval

def evaluate_sentence(gen_path, ref_path):
    gts = {}
    res = {}
    with open(ref_path, 'r') as f:
        gts_lines = f.readlines()
    with open(gen_path, 'r') as f:
        res_lines = f.readlines()

    for gts_line, res_line in zip(gts_lines, res_lines):
        sample = json.loads(gts_line.strip())
        generation = json.loads(res_line.strip())
        key = '#'.join(sorted(sample['entities']))
        if key not in gts:
            gts[key] = []
            gts[key].append(sample['text'])
            res[key] = []
            res[key].append(generation['text'])
        else:
            gts[key].append(sample['text'])
    return evaluator(gts, res)

def evaluate_story(gen_path, ref_path):
    gts = {}
    res = {}
    with open(ref_path, 'r') as f:
        gts_lines = f.readlines()
    with open(gen_path, 'r') as f:
        res_lines = f.readlines()

    for gts_line, res_line in zip(gts_lines, res_lines):
        sample = json.loads(gts_line.strip())
        generation = json.loads(res_line.strip())
        key = sample['id']
        gts[key] = []
        gts[key].append(sample['text'])
        res[key] = []
        res[key].append(generation['text'])
    return evaluator(gts, res)


def parse_text_to_tokens(sentence):
    return [t.text for t in nlp(sentence)]

def compute_recall(concepts, prediction, verbose=False):
    concept_token_list = parse_text_to_tokens(concepts)
    cnt = 0.
    for t in concept_token_list:
        if prediction.find(t) != -1:
            cnt += 1

    recall = cnt / len(concept_token_list)
    if verbose:
        print(f'concepts:\t{concepts}\nprediction:\t{prediction}\n'
            f'recall rate = {recall*100:.2f} %')
    return recall

def avg_list(lst):
    if not lst:
        return 0
    return sum(lst) / len(lst)

def compute_concept_recall(concept_list, pred_list):
    recall_list = []
    for concepts, prediction in zip(concept_list, pred_list):
        recall_list.append(compute_recall(concepts, prediction))
    recall_rate = avg_list(recall_list)
    return recall_rate


def ngram_precook(s, n=4, verbose=False):
    words = s.split()
    repeat_index = defaultdict(int)
    for k in range(1,n+1):  # k gram
        all_k_gram_list = []
        for i in range(len(words)-k+1):  # start index
            ngram = tuple(words[i:i+k])
            all_k_gram_list.append(ngram)
        num_unique_k_gram = len(set(all_k_gram_list))
        if len(all_k_gram_list):
            repeat_index[k] = float(num_unique_k_gram) / len(all_k_gram_list)
        else:  # no k-gram
            repeat_index[k] = 1e-13
        if verbose:
            print(f'{k}-gram:\tindex:\t{repeat_index[k]:.2f}\tall:\t{len(all_k_gram_list)}\tunique:{num_unique_k_gram}')
    return repeat_index


def compute_repetition(text_list, n=4, verbose=False):
    """
    See https://arxiv.org/pdf/2202.06417.pdf Section 4.1.2
    """
    repetition_list = defaultdict(list)
    for text in text_list:
        repeat_score = ngram_precook(text, n=n, verbose=verbose)
        for k in range(1, n+1):
            repetition_list[k].append(repeat_score[k])
    repetition_scores = {k: 1 - np.mean(repetition_list[k]) for k in range(1, n+1)}
    if verbose:
        for k in range(1, n+1):
            print(f'rep-{k}:\t{repetition_scores[k]}\t{repetition_list[k]}\tmean:({np.mean(repetition_list[k])})')
    return repetition_scores


def compute_diversity(repetition_scores):
    """
    See https://arxiv.org/pdf/2202.06417.pdf Section 4.1.2
    """
    diversity = 1.
    for k in range(2, 5):
        diversity *= (1-repetition_scores[k])
    return diversity


def distinct_n_sentence_level(sentence, n):
    """
    Compute distinct-N for a single sentence.
    :param sentence: a list of words.
    :param n: int, ngram.
    :return: float, the metric value.
    """
    if len(sentence) == 0:
        return 0.0  # Prevent a zero division
    distinct_ngrams = set(nltk.ngrams(sentence, n))
    return len(distinct_ngrams) / len(sentence)


def distinct_n_corpus_level(sentences, n):
    """
    Compute average distinct-N of a list of sentences (the corpus).
    :param sentences: a list of sentence.
    :param n: int, ngram.
    :return: float, the average value.
    """
    return sum(distinct_n_sentence_level(sentence, n) for sentence in sentences) / len(sentences)


def compute_distinct_n(text_list, n=2):
    distinct_n = {}
    for i in range(1, n+1):
        d = distinct_n_corpus_level(sentences=text_list, n=i)
        distinct_n[i] = d
    return distinct_n


In [67]:
def eval_text_list(text_list):
    metrics = {}
    repetition_scores = compute_repetition(text_list=text_list)
    for k, s in repetition_scores.items():
        metrics[f'rep-{k}'] = s

    diversity_scores = compute_diversity(repetition_scores)
    metrics['diversity'] = diversity_scores

    distinct_scores = compute_distinct_n(text_list=text_list)
    for k in range(1, 3):
        metrics[f'distinct-{k}'] = distinct_scores[k]
    return metrics

In [81]:
%%time
print("P0.80")
p080_result = eval_text_list(p080_text)
print("P0.85")
p085_result = eval_text_list(p085_text)
print("P0.90")
p090_result = eval_text_list(p090_text)
print("P0.95")
p095_result = eval_text_list(p095_text)
print("P0.99")
p099_result = eval_text_list(p099_text)
print("Human")
human_result = eval_text_list(real_text)

P0.80
P0.85
P0.90
P0.95
P0.99
Human
CPU times: total: 4.05 s
Wall time: 4.05 s


In [82]:
combined = {
    "p080":p080_result,
    "p085":p085_result,
    "p090":p090_result,
    "p095":p095_result,
    "p099":p099_result,
    "human":human_result
}

In [90]:
print(pd.DataFrame(combined).T.drop(columns=["rep-1", "distinct-1"]).round(3).to_latex())

\begin{tabular}{lrrrrr}
\toprule
{} &  rep-2 &  rep-3 &  rep-4 &  diversity &  distinct-2 \\
\midrule
p080  &  0.120 &  0.090 &  0.072 &      0.743 &       0.650 \\
p085  &  0.095 &  0.073 &  0.063 &      0.786 &       0.689 \\
p090  &  0.072 &  0.055 &  0.047 &      0.835 &       0.722 \\
p095  &  0.054 &  0.044 &  0.039 &      0.868 &       0.760 \\
p099  &  0.040 &  0.034 &  0.032 &      0.897 &       0.783 \\
human &  0.021 &  0.013 &  0.012 &      0.954 &       0.870 \\
\bottomrule
\end{tabular}



C:\Users\Gamer\AppData\Local\Temp\ipykernel_25780\1624173298.py:1: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(pd.DataFrame(combined).T.drop(columns=["rep-1", "distinct-1"]).round(3).to_latex())
